## Intro
Quarto can render source from compositions of Markdown and inputs and outputs from code blocks in R, Python and Julia.

It can produce:

- Html, pdf, MS Word and Markdown documents
- presentations: revealjs, ppt, latex/beamer
- Websites
- Books

The source documents can be written in any editor, but Jupyter and VS Code are particularly suited for Python, RStudio for R

## Workflow
1. Execute all cells in the current notebook
2. Open a terminal
3. render the notebook to a couple of formats:

``` {.bash filename="Terminal"}
quarto render 09_02_quarto.ipynb --to html
quarto render 09_02_quarto.ipynb --to revealjs
```
When you render a Jupyter notebook with Quarto, the contents of the notebook (code, markdown, and outputs) are converted to plain markdown and then processed by Pandoc, which creates the finished format.

![](https://quarto.org/docs/get-started/hello/images/ipynb-how-it-works.png)

## YAML Options
Configuration of the rendering
``` {.bash filename="Terminal"}
format: 
  html:
    code-fold: false
```

change `code-fold` to `true` ...

## The Markdown Syntax
Quarto is based on Pandoc and uses its variation of markdown as its underlying document syntax.
https://quarto.org/docs/authoring/markdown-basics.html



## Example visualisation

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv

In [2]:
from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Slider
from bokeh.models import CustomJS
from bokeh.palettes import Spectral6
from bokeh.layouts import row, column

In [3]:
from bokeh.models import HoverTool, LassoSelectTool, Plot, WheelZoomTool, CrosshairTool, ResetTool, PanTool

In [4]:
output_notebook(hide_banner=True)

In [5]:
# Load the data:

In [6]:
load_dotenv(find_dotenv(usecwd=True));

In [7]:
data_folder=os.getenv('DATA_FOLDER')

In [8]:
fp = os.path.join(data_folder, 'co2_gdp', 'co2_gdp_country.csv')

In [9]:
data = pd.read_csv(fp)
data.head();

In [10]:
regions_list = data.region.unique().tolist()
color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)

In [11]:
#| fig-cap: "Visualisation created with Python bokeh"
source = ColumnDataSource(data[data['year']==1964])

hover = HoverTool(tooltips=[('Country', '@country'), ('GDP', '@gdp'), ('CO2 Emission', '@co2')])

# Create the figure: plot
p = figure(title='CO2 Emissions vs GDP in 1964', 
           height=400, width=750,
           # x_range=(xmin, xmax),
           # y_range=(ymin, ymax),
           y_axis_type='log',
           tools=[hover, WheelZoomTool(), PanTool()], # LassoSelectTool(), Plot(), WheelZoomTool(), CrosshairTool(), ResetTool() etc..
           toolbar_location="right",
           # toolbar_sticky=True # if the toolbar clashes with the axes
          )

# Add circle glyphs to the plot
p.circle(x='gdp', y='co2', fill_alpha=0.8, source=source, 
            legend_group='region',
            color=dict(field='region', transform=color_mapper),
            size=7)

p.legend.title = 'Region:'
# Set the legend.location attribute of the plot
p.legend.location = 'bottom_right'

# Set the x-axis label
p.xaxis.axis_label = 'GDP'

# Set the y-axis label
p.yaxis.axis_label = 'CO2 Emissions (tons per person)'

# Set autohide to true to only show the toolbar when mouse is over plot
p.toolbar.autohide = True

show(p)

## Slider with a JS callback

In [12]:
source = ColumnDataSource(data)

In [13]:
Curr = ColumnDataSource(data[data['year'] == 1964])

In [14]:
callback = CustomJS(args=dict(source=source, sc=Curr), code="""
        const yr = cb_obj.value;
        sc.data['gdp'] = [];
        sc.data['co2'] = [];
        sc.data['country'] = [];
        sc.data['region'] = [];
        sc.data['year'] = [];
        
        for (var i = 0; i <= source.get_length(); i++){
          if (source.data['year'][i] == yr){
                sc.data['gdp'].push(source.data['gdp'][i]);
                sc.data['co2'].push(source.data['co2'][i]);
                sc.data['country'].push(source.data['country'][i]);
                sc.data['region'].push(source.data['region'][i]);
                sc.data['year'].push(yr);
          }
        }
        
        sc.change.emit();
        
    """)

In [15]:
hover = HoverTool(tooltips=[('Country', '@country'), ('GDP', '@gdp'), ('CO2 Emission', '@co2')])

In [16]:
# Create the figure: plot
p = figure(title='CO2 Emissions vs GDP', 
           height=350, width=800,
           y_axis_type='log',
           tools=[hover, WheelZoomTool(), PanTool()], # LassoSelectTool(), Plot(), WheelZoomTool(), CrosshairTool(), ResetTool() etc..
           toolbar_location="right"
          )

# Add circle glyphs to the plot
p.circle(x='gdp', y='co2', fill_alpha=0.8, source=Curr, 
            legend_group='region',
            color=dict(field='region', transform=color_mapper),
            size=7)

# p.xaxis.ticker.desired_num_ticks = 10

p.legend.title = 'Region:'
# Set the legend.location attribute of the plot
p.legend.location = 'bottom_right'

# Set the x-axis label
p.xaxis.axis_label = 'GDP'

# Set the y-axis label
p.yaxis.axis_label = 'CO2 Emissions (tons per person)'

# Set autohide to true to only show the toolbar when mouse is over plot
p.toolbar.autohide = False
p.toolbar.active_drag = None
p.toolbar.active_inspect = None

slider = Slider(start=min(data.year), end=max(data.year), step=1, value=min(data.year), title='Year')
slider.js_on_change('value', callback)

layout = column(slider, p, sizing_mode='scale_both')
output_notebook(hide_banner=True)
show(layout)

## Initialise a git repository

* go to your zhaw github account
* create a new repository 
* follow the stops to initialise git repository locally
* push the files from local into the remote repository

## Deploy html docs on github pages {.smaller}
On github.zhaw.ch/your_username/your_new_repository

* go to Settings - Pages:
* select "Deploy from a branch": main
* and the root-directory
* within a minute your html document should be served as<br>
github.zhaw.ch/pages/your_username/your_new_repository/your_filename.html